# Playground for ASFPy module development

In [114]:
import os

from asfpy import asfpy

In [115]:
categories = [
    "Clinical",
    "Developmental",
    "Affective",
    "Social",
    "Cognitive"
]

In [116]:
applicants = [
    {
        "id": "APP001",
        "urm": False,
        "lim": False,
        "du": True,
        "categories": {
            "Clinical"
        },
    },
    {
        "id": "APP002",
        "urm": True,
        "lim": False,
        "du": True,
        "categories": {
            "Cognitive"
        },
    },
    {
        "id": "APP003",
        "urm": True,
        "lim": False,
        "du": True,
        "categories": {
            "Affective",
            "Cognitive"
        }
    },
    {
        "id": "APP004",
        "urm": True,
        "lim": True,
        "du": False,
        "categories": {
            "Clinical"
        }
    },
    {
        "id": "APP005",
        "urm": False,
        "lim": True,
        "du": False,
        "categories": {
            "Developmental"
        }
    }
]

In [117]:
editors = [
    {
        "id": "EDI001",
        "role": "faculty",
        "categories": {
            "Clinical",
            "Developmental"
        },
        "capacity": 3
    },
    {
        "id": "EDI002",
        "role": "faculty",
        "categories": {
            "Affective"
        },
        "capacity": 2
    },
    {
        "id": "EDI003",
        "role": "faculty",
        "categories": {
            "Cognitive",
            "Affective"
        },
        "capacity": 15
    },
    {
        "id": "EDI004",
        "role": "student",
        "categories": {
            "Clinical",
            "Developmental"
        },
        "capacity": 5
    },
    {
        "id": "EDI005",
        "role": "student",
        "categories": {
            "Cognitive",
            "Social"
        },
        "capacity": 8
    },
        {
        "id": "EDI006",
        "role": "student",
        "categories": {
            "Clinical",
            "Developmental"
        },
        "capacity": 5
    }
]

In [118]:
prioritized_applicants = asfpy.prioritize(applicants);

In [119]:
faculty_editors = asfpy.editors_by_role(editors, "faculty")
student_editors = asfpy.editors_by_role(editors, "student")

In [120]:
asfpy.editors_by_categories(faculty_editors, {"Cognitive"})

[{'id': 'EDI003',
  'role': 'faculty',
  'categories': {'Affective', 'Cognitive'},
  'capacity': 15}]

In [121]:
import operator

def match(applicants, editors):
    
    matches = []
    
    for applicant in applicants:
        
        potential_editors = asfpy.editors_by_categories(editors, applicant["categories"])
        
        if asfpy.capacity(potential_editors) == 0:
        # If the editing capacity for an applicant is 0, continue to next applicant    
            continue
        else:
            faculty_editors = asfpy.editors_by_role(potential_editors, "faculty")
            student_editors = asfpy.editors_by_role(potential_editors, "student")
        
            if asfpy.capacity(faculty_editors) > 0:
                
                faculty_capacity = sorted([{
                    "capacity": asfpy.capacity(asfpy.editors_by_categories(faculty_editors, {c})), 
                    "category": c
                } for c in applicant["categories"]], key = operator.itemgetter("capacity"), reverse = True)
                
                # hc = highest capacity category given two options
                hc = faculty_capacity[0]["category"]
                
                faculty_in_category = sorted(
                    asfpy.editors_by_categories(faculty_editors, {hc}), 
                    key = operator.itemgetter("capacity"),
                    reverse = True
                )
                
                faculty_id = faculty_in_category[0]["id"]
                
                matches.append({
                    "applicant": applicant["id"],
                    "faculty": faculty_id
                })
                
                for e in editors:
                    if e["id"] == faculty_id:
                        e["capacity"] -= 1
                
            if asfpy.capacity(student_editors) > 0:               
                
                pass
            
    return matches

In [126]:
match(applicants, editors)

[{'applicant': 'APP002', 'faculty': 'EDI003'},
 {'applicant': 'APP003', 'faculty': 'EDI003'}]

In [125]:
editors

[{'id': 'EDI001',
  'role': 'faculty',
  'categories': {'Clinical', 'Developmental'},
  'capacity': 0},
 {'id': 'EDI002',
  'role': 'faculty',
  'categories': {'Affective'},
  'capacity': 2},
 {'id': 'EDI003',
  'role': 'faculty',
  'categories': {'Affective', 'Cognitive'},
  'capacity': 13},
 {'id': 'EDI004',
  'role': 'student',
  'categories': {'Clinical', 'Developmental'},
  'capacity': 5},
 {'id': 'EDI005',
  'role': 'student',
  'categories': {'Cognitive', 'Social'},
  'capacity': 8},
 {'id': 'EDI006',
  'role': 'student',
  'categories': {'Clinical', 'Developmental'},
  'capacity': 5}]